## Falcon 9 Launch Site Locations
#### Here, we'll be reviewing the locations of launch sites for the Falcon 9 and creating some maps via Folium to allow us to more easily utilize our data and get a sense for each launch site and their correlated successes and failues to land. 

We'll start by importing Folium and Pandas as well as a few additional plugins for Folium. Overall, our end goal here is to create a map that can be interacted with for sharing with interested parties. 

In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [7]:
# Download and read the `spacex_launch_geo.csv`
import requests
import io

# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(URL)
spacex_csv_file = io.BytesIO(response.content)
spacex_df = pd.read_csv(spacex_csv_file)

Starting with our DataFrame, we'll begin by filtering the DF down to the core components we need, including Launch Site, Latitute Coordinates, Longitude Coordinates, and the Class. As a reminder from previous components of this analysis, the "Class" column is where we're storing the values to verify whether a landing was successful or not, indicated by either a 0, for a failure, or a 1, for a success. 

In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Initializing our map, we'll set the NASA Space Center southeast of Houston as our starting point. Because rocket launches need to occur closer to the equator for a variety of reasons, the location of the Houston NASA Space Center seems like a logical place to center our map for this portion of our analysis. 

In [9]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now that we have our map created, we are going to be adding in the relevant Python code we need to add the Launch sites from our Dataframe utilizing their Latitude and Longitude coordinates. We will add this in, and include our Class later on to include the successes and failures. 


In [11]:
# Creating and adding folium.circle and folium.marker for each launch site. 
for index, row in launch_sites_df.iterrows():
    # Grabbing Coordinates for each launch site.
    coordinate = [row['Lat'], row['Long']]
    #Creating circle for launch site.
    launch_circle = folium.Circle(
        coordinate,
        radius = 1000, 
        color='#d35400', 
        fill=True
    ).add_child(folium.Popup(row['Launch Site']))

    launch_marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor = (0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % row['Launch Site'],
        )
    )

    site_map.add_child(launch_circle)
    site_map.add_child(launch_marker)

site_map

Here is where we iterate through the class row to determine if we had a successful landing, and if we did not, specifying what we would like the marker to be show as. Additionally, we coorelate each class to the corresponding latitude and longitude of it's respectful row. 


In [12]:
# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for each launch in spacex_df to the cluster
for index, row in spacex_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    if row['class'] == 1:
        marker_color = 'green'
        outcome = 'Success'
    else:
        marker_color = 'red'
        outcome = 'Failure'
    folium.Marker(
        location=coordinate,
        popup=f"{row['Launch Site']} - {outcome}",
        icon=folium.Icon(color=marker_color, icon='cloud', prefix='fa')
    ).add_to(marker_cluster)

# Display the map
site_map

Here, we're adding in the logic to show the latitude and longitude in the top right corner based on the position of our mouse. We can interact with the map and see the location on the world based on our mouses position. 

In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Additionally, we want to create the logic for calculating the distance

In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

coastline_points = [
    {'name': 'Cape Canaveral Coast', 'lat': 28.56367, 'lon': -80.57163},
    {'name': 'Kennedy Space Center', 'lat': 28.57276, 'lon': -80.57047},
    {'name': 'VAFB', 'lat': 34.63413, 'lon': -120.62525}
]
# Initialize a MarkerCluster for coastline points
coastline_cluster = MarkerCluster(name='Coastline Points').add_to(site_map)

# For each launch site, find the closest coastline point and mark it
for index, row in launch_sites_df.iterrows():
    launch_site_lat = row['Lat']
    launch_site_lon = row['Long']
    launch_site_name = row['Launch Site']
    
    # Find the closest coastline point
    min_distance = float('inf')
    closest_coastline = None
    for coast in coastline_points:
        distance = calculate_distance(launch_site_lat, launch_site_lon, coast['lat'], coast['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast
    
    # Print the result
    print(f"Launch Site: {launch_site_name}")
    print(f"Closest Coastline: {closest_coastline['name']} (Lat: {closest_coastline['lat']}, Lon: {closest_coastline['lon']})")
    print(f"Distance to Coastline: {min_distance:.2f} km")
    
    # Add a marker for the closest coastline point
    folium.Marker(
        location=[closest_coastline['lat'], closest_coastline['lon']],
        popup=f"{closest_coastline['name']} - {launch_site_name} ({min_distance:.2f} km)",
        icon=folium.Icon(color='purple', icon='water', prefix='fa')
    ).add_to(coastline_cluster)
    
    # Add a line connecting the launch site to the closest coastline
    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [closest_coastline['lat'], closest_coastline['lon']]],
        color='purple',
        weight=2,
        popup=f"Distance to {closest_coastline['name']}: {min_distance:.2f} km"
    ).add_to(site_map)

# Add layer control to toggle coastline markers
folium.LayerControl().add_to(site_map)

# Display the map
site_map

Launch Site: CCAFS LC-40
Closest Coastline: Cape Canaveral Coast (Lat: 28.56367, Lon: -80.57163)
Distance to Coastline: 0.58 km
Launch Site: CCAFS SLC-40
Closest Coastline: Cape Canaveral Coast (Lat: 28.56367, Lon: -80.57163)
Distance to Coastline: 0.51 km
Launch Site: KSC LC-39A
Closest Coastline: Cape Canaveral Coast (Lat: 28.56367, Lon: -80.57163)
Distance to Coastline: 7.43 km
Launch Site: VAFB SLC-4E
Closest Coastline: VAFB (Lat: 34.63413, Lon: -120.62525)
Distance to Coastline: 1.34 km


In [16]:
# Define coastline points
coastline_points = [
    {'name': 'Cape Canaveral Coast', 'lat': 28.56367, 'lon': -80.57163},
    {'name': 'Vandenberg Coast', 'lat': 34.632, 'lon': -120.626},
]

# For each launch site, find the closest coastline and add a PolyLine
for index, row in launch_sites_df.iterrows():
    launch_site_lat = row['Lat']
    launch_site_lon = row['Long']
    launch_site_name = row['Launch Site']
    
    # Find the closest coastline point
    min_distance = float('inf')
    closest_coastline = None
    for coast in coastline_points:
        distance = calculate_distance(launch_site_lat, launch_site_lon, coast['lat'], coast['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_coastline = coast
    
    # Create coordinates list for PolyLine
    coordinates = [[launch_site_lat, launch_site_lon], [closest_coastline['lat'], closest_coastline['lon']]]
    
    # Create and add PolyLine
    lines = folium.PolyLine(
        locations=coordinates,
        weight=1,
        color='purple',
        popup=f"{launch_site_name} to {closest_coastline['name']} ({min_distance:.2f} km)"
    )
    site_map.add_child(lines)

# Display the map
site_map

In [17]:
# Define points for cities, railways, highways, etc. (replace with your own coordinates)
proximity_points = [
    {'name': 'Cape Canaveral City', 'type': 'City', 'lat': 28.3922, 'lon': -80.6077},  # Near CCAFS
    {'name': 'Vandenberg Railway', 'type': 'Railway', 'lat': 34.582, 'lon': -120.626},  # Near VAFB
    {'name': 'Highway 1', 'type': 'Highway', 'lat': 34.632, 'lon': -120.610},  # Near VAFB
]

# For each launch site, find the closest proximity point and add marker and line
for index, row in launch_sites_df.iterrows():
    launch_site_lat = row['Lat']
    launch_site_lon = row['Long']
    launch_site_name = row['Launch Site']
    
    # Find the closest proximity point
    min_distance = float('inf')
    closest_point = None
    for point in proximity_points:
        distance = calculate_distance(launch_site_lat, launch_site_lon, point['lat'], point['lon'])
        if distance < min_distance:
            min_distance = distance
            closest_point = point
    
    # Create marker for the closest point
    folium.Marker(
        location=[closest_point['lat'], closest_point['lon']],
        popup=f"{closest_point['name']} ({closest_point['type']}) - {launch_site_name} ({min_distance:.2f} km)",
        icon=folium.Icon(color='blue', icon='info-sign', prefix='fa')
    ).add_to(site_map)
    
    # Create coordinates list for PolyLine
    coordinates = [[launch_site_lat, launch_site_lon], [closest_point['lat'], closest_point['lon']]]
    
    # Create and add PolyLine
    lines = folium.PolyLine(
        locations=coordinates,
        weight=1,
        color='blue',
        popup=f"{launch_site_name} to {closest_point['name']} ({min_distance:.2f} km)"
    )
    site_map.add_child(lines)

# Display the map
site_map